In [42]:
import os
from dotenv import load_dotenv
from operator import itemgetter
import glob
load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

MODEL = "gpt-3.5-turbo"
MODEL = "mixtral:8x7b"
MODEL = "llama3.1:8b"

In [37]:
if OPEN_API_KEY is None:
    raise ValueError("API key not found. Make sure .env file is present and properly configured.")


In [50]:
from glob import glob
from langchain_community.document_loaders import PyMuPDFLoader

# List of PDF file paths
pdf_files = glob('Documents/*.pdf')

# Initialize an empty list to hold all pages
all_pages = []

# Iterate over each PDF file
for pdf_file in pdf_files:
    loader = PyMuPDFLoader(pdf_file)
    pages = loader.load_and_split()
    all_pages.extend(pages)  # Add the pages to the list

# Now `all_pages` contains the pages from all PDF files


In [52]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader

template = """
Answer the question bases on the context below. if you can't answer the question , replay "I don't know what you meant."

context : {context}
Question: {question}
"""
prompt = PromptTemplate.from_template(template)
prompt.format(context="here is the some context ",
              question = "here is the question")
# 
# loader = PyMuPDFLoader(pdf_files)
# pages = loader.load_and_split()


if MODEL.startswith('gpt'):
    model = ChatOpenAI(
        api_key=OPEN_API_KEY,
        model=MODEL
    )
    embeddings  = OpenAIEmbeddings(
        api_key=OPEN_API_KEY
    )
else:
    model = Ollama(
        model = MODEL
    )
    embeddings = OllamaEmbeddings(
        model = MODEL
    )

from langchain_community.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(all_pages, embeddings)
retriever = db.as_retriever()

    
parser = StrOutputParser()
chain = prompt| model | parser

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question") ,
    }| 
    prompt | model | parser
    
)

In [54]:
from operator import itemgetter
questions = [
    "what is my name?",
    "what is my email address?"
    "what is docker?"
]
for question in questions:
    print("Question:", question)
    answer = chain.invoke(
    {
        "question": question
    }
    )
    print("Answer: {answer}".format(answer=answer))



Question: what is my name?
Answer: I don't know what you meant.
Question: what is my email address?what is docker?
Answer: Two separate questions!

For the first question:
Your email address is iffishells@gmail.com.

And for the second question:
Docker is a tool designed to make it easier to create and deploy, and run applications by using containers. Containers allow a developer to package an application and its dependencies in a single file, called an image, which can be easily run on any system that has Docker installed, without requiring manual installation of libraries or frameworks.


'Muhammad iftikhar'

In [20]:
# retriever.invoke("what is openai")

'Muhammad iftikhar'